#**DataSet**

In [3]:
from astropy.wcs.docstrings import axis_types

pip install datasets transformers evaluate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import datasets
import transformers
import evaluate
import numpy as np
import torch # 파이토치

In [5]:
데이터 = datasets.load_dataset('imdb')
데이터

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
데이터['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [7]:
type(데이터['train'])

datasets.arrow_dataset.Dataset

In [8]:
import numpy as np

# Select the desired columns and number of rows
num_rows = 10  # Number of rows to select
selected_columns = ['text', 'label']  # Columns to select

# Convert the selected data to a NumPy array
numpy_data = 데이터['train'].select(range(num_rows)).to_pandas()[selected_columns].to_numpy()

# Print the NumPy array
print(numpy_data)

[['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, 

In [9]:
datasets.Dataset.from_dict?

In [10]:
train_dict = {
    'text': [
        "I prefer making decisions based on logic and objective facts.",
        "I always consider how others might feel when making a decision.",
        "Data and analysis drive most of my decisions.",
        "I rely on my empathy and personal values to guide my choices."
    ],
    'label': [0, 1, 0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

test_dict = {
    'text': [
        "I find it important to weigh all the pros and cons logically.",
        "When making decisions, I prioritize harmony and people's emotions."
    ],
    'label': [0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

In [11]:
train_dict

{'text': ['I prefer making decisions based on logic and objective facts.',
  'I always consider how others might feel when making a decision.',
  'Data and analysis drive most of my decisions.',
  'I rely on my empathy and personal values to guide my choices.'],
 'label': [0, 1, 0, 1]}

In [16]:
datasets.Dataset.from_dict(train_dict)

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [17]:
데이터['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

#데이터['train'] 과 datasets.Dataset.from_dict(train_dict) 비교
- 데이터['train'] 과 datasets.Dataset.from_dict(train_dict) 모두 Dataset 객체를 반환하며 출력 형식이 비슷합니다:    


 Dataset({
    features: ['text', 'label'],
    num_rows: 25000  # datasets.Dataset.from_dict(train_dict)의 경우 4
})


- 이를 ['test']에도 적용해보자.


In [18]:
train_data = datasets.Dataset.from_dict(train_dict)
test_data = datasets.Dataset.from_dict(test_dict)

In [19]:
datasets.dataset_dict.DatasetDict?

결과:  Init signature: datasets.dataset_dict.DatasetDict(*args, **kwargs)
Docstring:      A dictionary (dict of str: datasets.Dataset) with dataset transforms methods (map, filter, etc.)
File:           /usr/local/lib/python3.10/dist-packages/datasets/dataset_dict.py
Type:           type




그냥, 결론은 datasets.dataset_dict.DatasetDict()




의 괄호 안에 'dic'형태로 입력값을 받는다.


밑의 코드에서는

{'train':train_data, 'test':test_data}

가 해당한다


In [20]:
나의데이터 = datasets.dataset_dict.DatasetDict({'train':train_data, 'test':test_data})

In [21]:
나의데이터

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})

In [22]:
나의데이터['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 4
})

In [23]:
나의데이터['train'][0]

{'text': 'I prefer making decisions based on logic and objective facts.',
 'label': 0}


#**토크나이저**
```
# 코드로 형식 지정됨
```



In [24]:
토크나이저 = 데이터전처리하기1 = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


C:\Users\qnsgh\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
토크나이저?

In [26]:
토크나이저('hi hello')

{'input_ids': [101, 7632, 7592, 102], 'attention_mask': [1, 1, 1, 1]}

In [27]:
토크나이저('hi ㅎㅇㅎㅇㅎㅇ ㅎㅇ hello')

{'input_ids': [101, 7632, 100, 100, 7592, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [28]:
토크나이저('hi hello hello hello')

{'input_ids': [101, 7632, 7592, 7592, 7592, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}

- 단어와 숫자 間 매핑 기능 수행 & [101], [102]는 처음과 끝을 알려줌

In [29]:
토크나이저(['hi hi', 'hello', 'hello hello hi'])

{'input_ids': [[101, 7632, 7632, 102], [101, 7592, 102], [101, 7592, 7592, 7632, 102]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1]]}

- 이렇게 콤마와 따옴표를 통해 병렬적으로(리스트) 구성할 수도 있음.

### truncation이란? --> 너무 문장이 길어지면 알아서 끊어줌. 최대 한도 512. <-> 너무 짧아서 실행이 안 되기도 할까?(0). max_length, padding과 관련이 있다.

In [30]:
len(토크나이저('hi hello'*2)['input_ids'])

6

## `len(토크나이저('hi hello' * 2))`와 `len(토크나이저('hi hello' * 2)['input_ids'])`의 차이

### 1. `len(토크나이저('hi hello' * 2))`
이 코드는 토크나이저가 반환하는 **전체 출력**의 길이를 확인하는 것입니다. Hugging Face의 토크나이저는 일반적으로 **딕셔너리**를 반환하며, 이 딕셔너리는 여러 가지 키를 포함할 수 있습니다. 대표적으로 다음과 같은 두 개의 주요 키가 있습니다:
- `input_ids`: 텍스트를 토큰화한 후 고유한 ID로 변환한 리스트.
- `attention_mask`: 각 토큰에 대한 attention을 적용할지 여부를 나타내는 마스크.

따라서 `len(토크나이저(...))`은 딕셔너리의 **키 개수**를 의미하며, 일반적으로 **2**가 됩니다. (※ 토크나이저는 **input_ids**와 **attention_mask**라는 두 가지 주요 정보를 반환하기 때문임.)

### 2. `len(토크나이저('hi hello' * 2)['input_ids'])`
이 코드는 토크나이저의 출력 중 **`input_ids`**의 길이를 확인합니다. `input_ids`는 텍스트 `"hi hello hi hello"`를 토큰화한 결과입니다.

- `"hi"`는 1개의 토큰으로 변환됩니다.
- `"hello"`도 1개의 토큰으로 변환됩니다.
  
따라서 `"hi hello hi hello"`는 총 4개의 토큰이 존재합니다. 여기에 모델을 위한 **시작 토큰** (`[CLS]`)과 **종료 토큰** (`[SEP]`)이 추가되므로 최종적으로 `input_ids`의 길이는 **6**이 됩니다.

### 요약
- **`len(토크나이저('hi hello' * 2))`**: 토크나이저가 반환하는 딕셔너리의 키 개수 (보통 2).
- **`len(토크나이저('hi hello' * 2)['input_ids'])`**: 텍스트를 토큰화한 후 추가된 토큰 포함 총 **6개의 토큰**이 생성됨.

---

## `input_ids`에서 `ids`의 의미

`input_ids`에서 **`ids`**는 "Identifiers"의 약자입니다. 즉, **토큰의 고유 식별자(IDs)**를 의미합니다.

토크나이저는 텍스트를 처리할 때, 텍스트를 **토큰화**하고, 각 토큰을 고유한 숫자 ID로 변환합니다. 모델은 이러한 숫자 ID를 처리합니다.

### 예시
- `"hello world"`라는 문장을 토크나이저가 처리하면:
  - `"hello"` → 토큰 ID: 7592
  - `"world"` → 토큰 ID: 2088
- 이렇게 토큰화된 후 `input_ids`는 `[7592, 2088]`과 같이 나타납니다.

### 요약
- **`input_ids`**: 텍스트가 토큰화되어 고유한 숫자 식별자로 변환된 결과.
- **`ids`**: 각 토큰을 식별하는 고유한 **숫자 식별자(Identifiers)**를 의미.


In [31]:
len(토크나이저('hi'*3)['input_ids'])

5

In [32]:
len(토크나이저('[hi]'*3)['input_ids'])

11

In [33]:
(토크나이저('[hi]'*3)['input_ids'])

[101, 1031, 7632, 1033, 1031, 7632, 1033, 1031, 7632, 1033, 102]

In [34]:
len(토크나이저(['hi']*3)['input_ids'])


3

In [35]:
len(토크나이저(['hi hi', 'hello']*300)['input_ids'])

600

In [36]:
len(토크나이저('hi hello '*300,truncation=False)['input_ids']) # 원래는 602가 나와야하는데 짤려서 512만 나옴

Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors


602

In [37]:
len(토크나이저('hi hello '*300,truncation=True)['input_ids']) # 원래는 602가 나와야하는데 짤려서 512만 나옴


512

In [38]:
len(토크나이저('hi hello '*300)['input_ids']) # truncation을 지웠을 때는 정상적으로 602 출력


602

# **인공지능 (중요)**

## 1단계

In [54]:
torch.manual_seed()  # 인공지능의 weight? 고정시키는 값.
인공지능 = model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
 인공지능.classifier.weight

Parameter containing:
tensor([[-0.0234,  0.0279,  0.0242,  ...,  0.0091, -0.0063, -0.0133],
        [ 0.0087,  0.0007, -0.0099,  ...,  0.0183, -0.0007,  0.0295]],
       requires_grad=True)

In [56]:
인공지능?

In [57]:
인공지능.pre_classifier.weight

Parameter containing:
tensor([[-0.0122,  0.0030,  0.0301,  ...,  0.0003,  0.0327, -0.0123],
        [-0.0041,  0.0008,  0.0169,  ..., -0.0163, -0.0117, -0.0135],
        [ 0.0020, -0.0215, -0.0021,  ...,  0.0016,  0.0102, -0.0483],
        ...,
        [-0.0026, -0.0327,  0.0099,  ...,  0.0341, -0.0184, -0.0109],
        [ 0.0088, -0.0345, -0.0011,  ...,  0.0018,  0.0172, -0.0122],
        [-0.0148,  0.0147, -0.0184,  ..., -0.0166,  0.0241,  0.0201]],
       requires_grad=True)

In [58]:
입력정보_원시텍스트 = "This movie was a huge disappointment."
정리된숫자_토큰화된자료 = 토크나이저(입력정보_원시텍스트,return_tensors='pt')
정리된숫자_토큰화된자료

{'input_ids': tensor([[  101,  2023,  3185,  2001,  1037,  4121, 10520,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [59]:
인공지능(**정리된숫자_토큰화된자료)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1173,  0.0261]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [60]:
인공지능(**정리된숫자_토큰화된자료).logits.detach()

tensor([[-0.1173,  0.0261]])

^^^ 는 tensor 값만 표현하는 것

In [48]:
계산된숫자_로짓 = 인공지능(**정리된숫자_토큰화된자료).logits.detach().numpy()
계산된숫자_로짓

array([[ 0.07220297, -0.05291047]], dtype=float32)

In [47]:
출력정보_확률 = np.exp(계산된숫자_로짓)/np.exp(계산된숫자_로짓).sum() # 0일확률(=부정평가일확률), 1일확률(=긍정평가일확률)
출력정보_확률

array([[0.5312376 , 0.46876237]], dtype=float32)

^^^ 더해서 1이 됨. (왜냐하면 exp의 공식의 성질 때문임. 확률과 비슷한 양상을 띰)

In [61]:
출력정보_확률.argmax() # 부정적 영화평가에 대한 인공지능의 예측.

0

In [62]:
계산된숫자_로짓.argmax() # 부정적 영화평가에 대한 인공지능의 예측. << 왜??

0

출력정보_확률은 exp 함수에 a와 b의 값을 각각 대입하여서 구한 값임.
계산된숫자_로짓은 just a와 b의 값을 비교한 것임.
exp함수는 증가하는 함수이므로 exp(a)-exp(b)의 비교와 a-b 間 대소 비교 결과는 동일할 것임.

두 가지의 인공지능의 계산 방식이 다르다면, 그 두가지 인공지능은 서로 다른 모델이라고 취급함.
출력값이 달라도 계산 방식이 동일하다면, 같은 인공지능이라고 취급.

## 2단계

#### 미니 배치의 이해

In [66]:
입력정보들_원시텍스트 = 데이터['train'][:16]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시텍스트,truncation=True,return_tensors='pt',padding=True)
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
출력정보들_확률

array([[0.47823825, 0.5217617 ],
       [0.47215328, 0.5278467 ],
       [0.48049477, 0.5195052 ],
       [0.46609667, 0.5339033 ],
       [0.4881449 , 0.51185507],
       [0.48004225, 0.5199578 ],
       [0.49288097, 0.50711906],
       [0.49470252, 0.5052974 ],
       [0.4941453 , 0.50585467],
       [0.490162  , 0.50983804],
       [0.495789  , 0.504211  ],
       [0.48260576, 0.51739424],
       [0.49386352, 0.5061365 ],
       [0.49013382, 0.50986624],
       [0.4824018 , 0.5175982 ],
       [0.49301967, 0.50698036]], dtype=float32)

기억할 것: 정리된숫자들_토큰화된자료 는 모두 길이가 512임. 너무 길면 512로 자르며, 너무 짧으면 0으로 땜빵(?)시킨다는 의미임. 단어가 존재하면 1, 존재하지 않으면 0으로 모두 채워버리는 기능 padding. (그리고 그렇게 되도록 패딩함. 정리된숫자들_토큰화된자료 는 Truncation과 Padding 기능이 결홥된 형태로 존재함.)


## 3단계

### 동적 배치의 이해

In [67]:
입력정보들_원시텍스트 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시텍스트,truncation=True,return_tensors='pt',padding=True)
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
출력정보들_확률

array([[0.47823825, 0.5217618 ],
       [0.47215328, 0.5278467 ],
       [0.48049477, 0.5195052 ],
       [0.46609667, 0.5339033 ]], dtype=float32)

Q. 만약에 배치 사이즈를 4로 줄인다면 어떨까? (평소에은 16 또는 32의 크기를 가짐.)

In [69]:
정리된숫자들_토큰화된자료['input_ids'] # 의 차원은 어떠할까? (4,512)일까?

tensor([[  101,  1045, 12524,  ...,  5436,  1012,   102],
        [  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  2065,  2069,  ...,     0,     0,     0],
        [  101,  2023,  2143,  ...,     0,     0,     0]])

^^^ (4,'512')네?

In [70]:
정리된숫자들_토큰화된자료['attention_mask']

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

^^^ attention_mask 또한 (4,363)이넹?

#### 1. 초기 데이터
- 입력 데이터는 `데이터['train'][:4]['text']`로, **처음 4개의 텍스트 데이터**를 사용합니다.

#### 2. attention_mask의 역할
- `attention_mask`는 **토큰화된 데이터**에서 유효한 토큰과 패딩(padding)을 구분하기 위해 사용됩니다.
- 유효한 토큰은 `1`, 패딩된 부분은 `0`으로 표시됩니다.

#### 3. sum(axis=1)의 결과
- `정리된숫자들_토큰화된자료['attention_mask'].sum(axis=1)`은 각 문장에서 **유효한 토큰의 개수**를 계산합니다.
- 결과: `[363, 304, 133, 185]`
  - 이는 각각의 문장에 있는 **유효한 토큰의 수**를 나타냅니다.
    - 첫 번째 문장은 363개의 유효 토큰
    - 두 번째 문장은 304개의 유효 토큰
    - 세 번째 문장은 133개의 유효 토큰
    - 네 번째 문장은 185개의 유효 토큰

#### 4. 패딩(padding)
- **가장 긴 문장**이 363개의 토큰이므로, 다른 문장들도 **363개의 토큰**에 맞춰 패딩을 추가합니다.
- 패딩은 더 짧은 문장의 길이를 맞추기 위해 0으로 채워주는 것을 의미합니다.

#### 5. 결론
- 각 문장의 길이를 맞추기 위해, 가장 긴 문장(363개의 토큰)에 맞춰 나머지 문장들도 패딩을 추가하여 **길이를 363개로 통일**합니다.


`-` 따라서 덩어리의 상태에 따라 (유)동적으로 배치 사이즈를 설정함.



## 4단계

### 손실(loss)의 개념을 이해하자.

In [108]:
입력정보들_원시텍스트 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시텍스트,truncation=True,return_tensors='pt',padding=True)
#데이터['train'][:4]['label'] # 정답확인
정리된숫자들_토큰화된자료['labels'] = torch.tensor([0,0,0,0]) # 정답입력
# ^^^ label을 학습시켜야 하니까 torch.tensor 의 형태로 키 입력
인공지능(**정리된숫자들_토큰화된자료)

SequenceClassifierOutput(loss=tensor(0.7461, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0847,  0.0024],
        [-0.0830,  0.0285],
        [-0.0600,  0.0180],
        [-0.0919,  0.0440]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

```Python
SequenceClassifierOutput **(loss=tensor(0.7461, grad_fn=<NllLossBackward0>)**, logits=tensor([[-0.0847,  0.0024],
        [-0.0830,  0.0285],
        [-0.0600,  0.0180],
        [-0.0919,  0.0440]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
```

지금껏 해왔던 것과 달리, **loss** 발생




`-` 정리해보자

In [105]:
'''CASE1'''


입력정보들_원시텍스트 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시텍스트,truncation=True,return_tensors='pt',padding=True)
print(f'실제정답: {데이터["train"][:4]["label"]}') # 정답확인
정리된숫자들_토큰화된자료['labels'] = torch.tensor([0,0,0,0]) # 정답입력
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()
출력정보들_확률 = np.exp(계산된숫자들_로짓)/np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1,1)
print(f'인공지능의예측: {계산된숫자들_로짓.argmax(axis=1)}')
print(f'인공지능의확신정도: {출력정보들_확률.max(axis=1)}')
print(f'손실(loss): {인공지능(**정리된숫자들_토큰화된자료).loss.item():.6f}')

실제정답: [0, 0, 0, 0]
인공지능의예측: [1 1 1 1]
인공지능의확신정도: [0.5217618 0.5278467 0.5195052 0.5339033]
손실(loss): 0.746100


In [106]:
'''CASE2'''


입력정보들_원시텍스트 = 데이터['train'][:4]['text']
정리된숫자들_토큰화된자료 = 토크나이저(입력정보들_원시텍스트, truncation=True, return_tensors='pt', padding=True)

# 정답 확인을 위한 출력
print(f'실제정답: {(데이터["train"][:4]["label"])}')

# 정답 입력
정리된숫자들_토큰화된자료['labels'] = torch.tensor(데이터['train'][12498:12502]['label'])  # 정답 입력

# AI 모델로 계산
계산된숫자들_로짓 = 인공지능(**정리된숫자들_토큰화된자료).logits.detach().numpy()

# 확률 계산
출력정보들_확률 = np.exp(계산된숫자들_로짓) / np.exp(계산된숫자들_로짓).sum(axis=1).reshape(-1, 1)

# AI 예측 결과 출력
print(f'인공지능의예측: {계산된숫자들_로짓.argmax(axis=1)}')
print(f'인공지능의확신정도: {출력정보들_확률.max(axis=1)}')

# 손실값 출력
print(f'손실(loss): {인공지능(**정리된숫자들_토큰화된자료).loss.item():.6f}')


실제정답: [0, 0, 0, 0]
인공지능의예측: [1 1 1 1]
인공지능의확신정도: [0.5217618 0.5278467 0.5195052 0.5339033]
손실(loss): 0.692629


In [107]:
인공지능(**정리된숫자들_토큰화된자료).loss

tensor(0.6926, grad_fn=<NllLossBackward0>)

CASE2가 좋긴 하군............

`-` 하지만 똑같이 틀려도 오답에 대한 확신(0.xxxxxxx)이 강할수록 loss가 크다.

`-` loss가 작으면 정답을 더 잘 맞히겠지만, 무조건 학습을 잘 하고 있다는 것을 의미하지 않음. (e.g., 열심히 문제 풀어서 오히려 답을 외워버린 것이나 미분의 의미를 정확히 이해하지 못 하고 무대뽀로 로피탈을 사용하는 것)